# Biterm topic modeling

We will use the [biterm plus library](https://bitermplus.readthedocs.io/en/latest/index.html) since it gives a practical implementation of the [biterm topic model](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.402.4032&rep=rep1&type=pdf). We will follow the steps of the tutorial written in its documentation, in order to extract the topics.

## Importations

In [1]:
import json
import pandas as pd
import re
from nltk.tokenize import TweetTokenizer
import string
import nltk.corpus
# nltk.download('stopwords')
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
import bitermplus as btm

## Tweets

### Data importation

First of all, we will import the clean tweets, just as we did in the previous notebook.

In [2]:
# Define companies' names and its products names
entities = ['Nintendo', 'Playstation', 'Xbox', 'Engage', 'Forspoken', 'HFRush']

In [3]:
# Import clean tweets
tweets = {}
for entity in entities:
    path = f'Videojocs/cleanTweets/{entity}.csv'
    tweets[entity] = pd.read_csv(path)
    tweets[entity] = tweets[entity].dropna() # Drop NaN values

### Prepare data for analysis

We have already tokenized the data in the previous section, so it only remains to convert the tokenized objects into a **corpus** (structured set of texts which we are working with) and dictionary.

In [4]:
# Vectorizing documents, obtaining full vocabulary and biterms
# Internally, btm.get_words_freqs uses CountVectorizer from sklearn
# You can pass any of its arguments to btm.get_words_freqs
texts = {}
X = {} # Documents vs words matrix in CSR format
vocabulary = {} # Vocabulary as a numpy.ndarray of terms
vocab_dict = {} # Vocabulary as a dictionary of {term: id} pairs
docs_vec = {} # Vectorised documents (list of numpy.ndarray objects with terms ids)
biterms = {} # List of biterms for each document
for entity in entities:
    texts[entity] = tweets[entity].lemmatizedText.tolist()
    X[entity], vocabulary[entity], vocab_dict[entity] = btm.get_words_freqs(texts[entity])
    docs_vec[entity] = btm.get_vectorized_docs(texts[entity], vocabulary[entity])
    biterms[entity] = btm.get_biterms(docs_vec[entity])

### BTM model training

We have to decide parameters, specially the number of topics that we want to deal with. Each topic is a combination of keywords and each keyword contributes a certain wightage to the topic. We make a first approach with just 10 topics and then we will tune the models.

In [5]:
model = {}
# Set number of topics (T) and number of top words for coherence calculation just as LDA in order to compare
T = 10
M = 30
# Initializing and running model
for entity in entities:
    model[entity] = btm.BTM(X[entity], vocabulary[entity], seed=12321, T=T, M=M)
    model[entity].fit(biterms[entity], iterations=20)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 12.02it/s]


#### Optimal number of topics

In order to estimate the optimal number of topics needed, we will calculate the **entropy** for various models and choose the one with the lower value. We will consider a number of topics between 2 and 25 (we discard the case of just 1 topic, since that would be always the obvious choice -the topic about the query made-).

In [6]:
entropy = {}
num_topics = {}
for entity in entities:
    entropy[entity] = btm.entropy(model[entity].matrix_topics_words_)
    num_topics[entity] = T

In [7]:
for entity in entities:
    print(f'The entropy of {entity} model considering {T} topics and {M} top words is {entropy[entity]}.')

The entropy of Nintendo model considering 10 topics and 30 top words is 3.6755964158061207.
The entropy of Playstation model considering 10 topics and 30 top words is 3.7040578800884925.
The entropy of Xbox model considering 10 topics and 30 top words is 3.949124809591601.
The entropy of Engage model considering 10 topics and 30 top words is 3.1631389240928347.
The entropy of Forspoken model considering 10 topics and 30 top words is 2.915112326899098.
The entropy of HFRush model considering 10 topics and 30 top words is 3.2241943868869565.


In [8]:
for entity in entities:
    for t in reversed(range(2,26)):
        modelTemp = btm.BTM(X[entity], vocabulary[entity], seed=12321, T=t, M=M)
        modelTemp.fit(biterms[entity], iterations=20)
        if (btm.entropy(modelTemp.matrix_topics_words_) < entropy[entity]): # This model with less topics has less entropy, so we keep it
            entropy[entity] = btm.entropy(modelTemp.matrix_topics_words_)
            model[entity] = modelTemp
            num_topics[entity] = t
            print(f'{entity} has now {entropy[entity]} entropy with {t} topics.')

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.47it/s]


Nintendo has now 3.671785345448951 entropy with 24 topics.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.56it/s]


Nintendo has now 3.641470749767549 entropy with 23 topics.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.61it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 11.31it/s]


HFRush has now 3.209878442624014 entropy with 13 topics.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 11.72it/s]


HFRush has now 3.198790619111692 entropy with 12 topics.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 31.40it/s]


In [9]:
num_topics

{'Nintendo': 15,
 'Playstation': 11,
 'Xbox': 12,
 'Engage': 11,
 'Forspoken': 7,
 'HFRush': 12}

### Inference

We will calculate documents vs topics probability matrix (make an inference).

In [10]:
p_zd = {}
for entity in entities:
    p_zd[entity] = model[entity].transform(docs_vec[entity])
# If you need to make an inference on a new dataset,
# you should vectorize it using your vocabulary from the training set:
#new_docs_vec = btm.get_vectorized_docs(new_texts, vocabulary)
#p_zd = model.transform(new_docs_vec)

100%|███████████████████████████████████████████████████████████████████████████| 7322/7322 [00:00<00:00, 53058.24it/s]


### Calculating metrics

To calculate perplexity, we must provide documents vs topics probability matrix that we calculated at the previous step.

In [11]:
perplexity = {} # How well a probability model predicts a sample (in NLP, degree of uncertainty a model has in predicting text)
coherence = {} # How well a topic is 'supported' by a text set (reference corpus)
# ALTERNATIVE FOR CALCULATION
#for entity in entities:
#perplexity[entity] = btm.perplexity(model[entity].matrix_topics_words_, p_zd[entity], X[entity], T)
#coherence[entity] = btm.coherence(model[entity].matrix_topics_words_, X[entity], M)
for entity in entities:
    perplexity[entity] = model[entity].perplexity_
    coherence[entity] = model[entity].coherence_
    print(f'Perplexity and coherence calculation for {entity} done.')

Perplexity and coherence calculation for Nintendo done.
Perplexity and coherence calculation for Playstation done.
Perplexity and coherence calculation for Xbox done.
Perplexity and coherence calculation for Engage done.
Perplexity and coherence calculation for Forspoken done.
Perplexity and coherence calculation for HFRush done.


In [12]:
# Visualize results
for entity in entities:
    print(f'{entity} perplexity: {perplexity[entity]}')
    print(f'{entity} coherence: {coherence[entity]}')

Nintendo perplexity: 713.8155557881406
Nintendo coherence: [-1436.0234493  -1693.96259335 -1579.08925461 -1793.019126
  -744.56780966 -1652.74434212 -1450.02869684 -1762.84515918
 -1691.84070316 -1516.22789225 -1557.37991458 -1085.26058952
 -1673.31470696 -1688.47069482 -1582.12338868]
Playstation perplexity: 326.9313794190241
Playstation coherence: [-1483.85975651 -1809.38994397 -1500.87444203 -2385.78841484
 -1642.200601   -1713.13050472 -1444.7498008  -1683.70569116
  -813.23459304 -1616.42737339 -1590.82013014]
Xbox perplexity: 167.90812034155223
Xbox coherence: [-1956.96578115 -1190.52849908 -1339.8770877  -1603.9220511
 -1536.88633287  -777.12111155 -1965.2590888  -1572.93900799
 -1808.93380327 -1530.94749933 -1755.57559094 -1767.25119235]
Engage perplexity: 539.5953974575604
Engage coherence: [-1378.41423494 -1350.51392755 -1353.27456224 -1297.23275051
 -1201.35160312 -1299.0082909  -1307.42292918 -1245.4756882
 -1444.58406329 -1425.02336855 -1429.5885136 ]
Forspoken perplexity:

### Visualizing results

Biterm plus only let us visualize the topic plots as a virtual machine instance, so we cannot store them and analyze them later. Thus, we will present all the plots for each game and company. However, we first need to do the importation of the visualization library.

In [13]:
import tmplot as tmp
import tomotopy as tmtp

In order get a fair comparison, we will need to set the function to calculate intertopic distance as **MDS (MultiDimensional Scaling)**.

#### Nintendo

In [14]:
# Run the interactive report interface
tmp.report(model=model['Nintendo'], docs=texts['Nintendo'])

#### Playstation

In [15]:
# Run the interactive report interface
tmp.report(model=model['Playstation'], docs=texts['Playstation'])

#### Xbox

In [16]:
# Run the interactive report interface
tmp.report(model=model['Xbox'], docs=texts['Xbox'])

#### Fire Emblem Engage

In [17]:
# Run the interactive report interface
tmp.report(model=model['Engage'], docs=texts['Engage'])

#### Forspoken

In [18]:
# Run the interactive report interface
tmp.report(model=model['Forspoken'], docs=texts['Forspoken'])

#### Hi-Fi Rush

In [19]:
# Run the interactive report interface
tmp.report(model=model['HFRush'], docs=texts['HFRush'])

### Model loading and saving

In [20]:
import pickle as pkl

# Saving
for entity in entities:
    filepath = f'Modelos/biterm/modelos/{entity}_{num_topics[entity]}.pkl'
    with open(filepath, "wb") as file:
        pkl.dump(model[entity], file)

## Reviews

### Data importation

First of all, we will import the clean reviews, just as we did in the previous notebook.

In [21]:
# Define games and platforms
games = [
    {'title': 'fire-emblem-engage', 'platform': 'switch', 'name': 'feSwitch'},
    {'title': 'hi-fi-rush', 'platform': 'xbox-series-x', 'name': 'hfrushXbox'},
    {'title': 'forspoken', 'platform': 'playstation-5', 'name': 'forspokenPS5'},
    {'title': 'hi-fi-rush', 'platform': 'pc', 'name': 'hfrushPc'},
    {'title': 'forspoken', 'platform': 'pc', 'name': 'forspokenPc'}
]

In [22]:
# Import reviews
reviews = {}
for game in games:
    user = pd.read_csv(f'Videojocs/cleanReviews/user_{game["name"]}.csv')
    scored = pd.read_csv(f'Videojocs/cleanReviews/scored_{game["name"]}.csv')
    unscored = pd.read_csv(f'Videojocs/cleanReviews/unscored_{game["name"]}.csv')
    frames = [user, scored, unscored]
    reviews[game["name"]] = pd.concat(frames)
    # Drop columns that are not necessary
    reviews[game["name"]] = reviews[game["name"]].drop(['source', 'link', 'date', 'grade', 'scoreType', 'upThumbs', 'totalThumbs', 'helpfulness'], axis=1)
    reviews[game["name"]] = reviews[game["name"]].dropna() # Drop NaN values

### Prepare data for analysis

We have already tokenized the data in the previous section, so it only remains to convert the tokenized objects into a **corpus** (structured set of texts which we are working with) and dictionary.

In [23]:
# Vectorizing documents, obtaining full vocabulary and biterms
# Internally, btm.get_words_freqs uses CountVectorizer from sklearn
# You can pass any of its arguments to btm.get_words_freqs
texts = {}
X = {} # Documents vs words matrix in CSR format
vocabulary = {} # Vocabulary as a numpy.ndarray of terms
vocab_dict = {} # Vocabulary as a dictionary of {term: id} pairs
docs_vec = {} # Vectorised documents (list of numpy.ndarray objects with terms ids)
biterms = {} # List of biterms for each document
for game in games:
    texts[game["name"]] = reviews[game["name"]].lemmatizedText.tolist()
    X[game["name"]], vocabulary[game["name"]], vocab_dict[game["name"]] = btm.get_words_freqs(texts[game["name"]])
    docs_vec[game["name"]] = btm.get_vectorized_docs(texts[game["name"]], vocabulary[game["name"]])
    biterms[game["name"]] = btm.get_biterms(docs_vec[game["name"]])

### BTM model training

We have to decide parameters, specially the number of topics that we want to deal with. Each topic is a combination of keywords and each keyword contributes a certain wightage to the topic. We make a first approach with just 10 topics and then we will tune the models.

In [24]:
model = {}
# Set number of topics (T) and number of top words for coherence calculation just as LDA in order to compare
T = 10
M = 30
# Initializing and running model
for game in games:
    model[game["name"]] = btm.BTM(X[game["name"]], vocabulary[game["name"]], seed=12321, T=T, M=M)
    model[game["name"]].fit(biterms[game["name"]], iterations=20)

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 131.58it/s]


#### Optimal number of topics

In order to estimate the optimal number of topics needed, we will calculate the **entropy** for various models and choose the one with the lower value. We will consider a number of topics between 2 and 25 (we discard the case of just 1 topic, since that would be always the obvious choice -the videogame that is being reviewed-).

In [25]:
entropy = {}
num_topics = {}
for game in games:
    entropy[game["name"]] = btm.entropy(model[game["name"]].matrix_topics_words_)
    num_topics[game["name"]] = T

In [26]:
for game in games:
    print(f'The entropy of {game["name"]} model considering {T} topics and {M} top words is {entropy[game["name"]]}.')

The entropy of feSwitch model considering 10 topics and 30 top words is 2.8897020786949246.
The entropy of hfrushXbox model considering 10 topics and 30 top words is 2.8622518449562424.
The entropy of forspokenPS5 model considering 10 topics and 30 top words is 2.7916286238464743.
The entropy of hfrushPc model considering 10 topics and 30 top words is 2.695077492240089.
The entropy of forspokenPc model considering 10 topics and 30 top words is 2.6462087495212145.


In [27]:
for game in games:
    for t in reversed(range(2,26)):
        modelTemp = btm.BTM(X[game["name"]], vocabulary[game["name"]], seed=12321, T=t, M=M)#, alpha=50/8, beta=0.01
        modelTemp.fit(biterms[game["name"]], iterations=20)
        if (btm.entropy(modelTemp.matrix_topics_words_) < entropy[game["name"]]): # This model with less topics has less entropy, so we keep it
            entropy[game["name"]] = btm.entropy(modelTemp.matrix_topics_words_)
            model[game["name"]] = modelTemp
            num_topics[game["name"]] = t
            print(f'{game["name"]} has now {entropy[game["name"]]} entropy with {t} topics.')

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 25.45it/s]


feSwitch has now 2.8674466250344146 entropy with 9 topics.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 47.62it/s]


hfrushXbox has now 2.8402542285130554 entropy with 9 topics.


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 45.98it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 151.52it/s]


forspokenPc has now 2.3096571615979298 entropy with 6 topics.


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 183.49it/s]


forspokenPc has now 2.227301244746888 entropy with 5 topics.


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 270.27it/s]


In [28]:
num_topics

{'feSwitch': 9,
 'hfrushXbox': 7,
 'forspokenPS5': 7,
 'hfrushPc': 5,
 'forspokenPc': 5}

### Inference

We will calculate documents vs topics probability matrix (make an inference).

In [29]:
p_zd = {}
for game in games:
    p_zd[game["name"]] = model[game["name"]].transform(docs_vec[game["name"]])
# If you need to make an inference on a new dataset,
# you should vectorize it using your vocabulary from the training set:
#new_docs_vec = btm.get_vectorized_docs(new_texts, vocabulary)
#p_zd = model.transform(new_docs_vec)

100%|█████████████████████████████████████████████████████████████████████████████| 159/159 [00:00<00:00, 26536.72it/s]


### Calculating metrics

To calculate perplexity, we must provide documents vs topics probability matrix that we calculated at the previous step.

In [30]:
perplexity = {} # How well a probability model predicts a sample (in NLP, degree of uncertainty a model has in predicting text)
coherence = {} # How well a topic is 'supported' by a text set (reference corpus)
# ALTERNATIVE FOR CALCULATION
#for entity in entities:
#perplexity[entity] = btm.perplexity(model[entity].matrix_topics_words_, p_zd[entity], X[entity], T)
#coherence[entity] = btm.coherence(model[entity].matrix_topics_words_, X[entity], M)
for game in games:
    perplexity[game["name"]] = model[game["name"]].perplexity_
    coherence[game["name"]] = model[game["name"]].coherence_
    print(f'Perplexity and coherence calculation for {game["name"]} done.')

Perplexity and coherence calculation for feSwitch done.
Perplexity and coherence calculation for hfrushXbox done.
Perplexity and coherence calculation for forspokenPS5 done.
Perplexity and coherence calculation for hfrushPc done.
Perplexity and coherence calculation for forspokenPc done.


In [31]:
# Visualize results
for game in games:
    print(f'{game["name"]} perplexity: {perplexity[game["name"]]}')
    print(f'{game["name"]} coherence: {coherence[game["name"]]}')

feSwitch perplexity: 799.6023795523932
feSwitch coherence: [-608.78049448 -629.78565814 -593.87784789 -644.40249464 -595.88650711
 -658.72324989 -654.51726294 -690.94633041 -655.77510323]
hfrushXbox perplexity: 526.4951238931591
hfrushXbox coherence: [-822.72724098 -837.89149172 -893.30083956 -931.89104425 -884.18353955
 -928.94171114 -857.04922181]
forspokenPS5 perplexity: 868.4850817177887
forspokenPS5 coherence: [-794.70696992 -736.19854299 -823.49531044 -758.16578719 -757.86717743
 -755.16623959 -695.13688279]
hfrushPc perplexity: 480.23031779663404
hfrushPc coherence: [-799.62184513 -755.16350576 -835.99182024 -781.03526879 -767.35257521]
forspokenPc perplexity: 637.5459214710432
forspokenPc coherence: [-680.60931442 -621.73887585 -646.77457301 -711.91102837 -736.30287702]


### Visualizing results

Biterm plus only let us visualize the topic plots as a virtual machine instance, so we cannot store them and analyze them later. Thus, we will present all the plots for each game and company. In order get a fair comparison, we will need to set the function to calculate intertopic distance as **MDS (MultiDimensional Scaling)**.

#### Fire Emblem Engage (Switch)

In [32]:
# Run the interactive report interface
tmp.report(model=model['feSwitch'], docs=texts['feSwitch'])

#### Forspoken (PC)

In [33]:
# Run the interactive report interface
tmp.report(model=model['forspokenPc'], docs=texts['forspokenPc'])

#### Forspoken (PS5)

In [34]:
# Run the interactive report interface
tmp.report(model=model['forspokenPS5'], docs=texts['forspokenPS5'])

#### Hi-Fi Rush (PC)

In [35]:
# Run the interactive report interface
tmp.report(model=model['hfrushPc'], docs=texts['hfrushPc'])

#### Hi-Fi Rush (Xbox Series X)

In [36]:
# Run the interactive report interface
tmp.report(model=model['hfrushXbox'], docs=texts['hfrushXbox'])

### Model loading and saving

In [37]:
import pickle as pkl

# Saving
for game in games:
    filepath = f'Modelos/biterm/modelos/{game["name"]}_{T}.pkl'
    with open(filepath, "wb") as file:
        pkl.dump(model[game["name"]], file)

## Next notebook

In this notebook we have deepened into the biterm topic model and the optimal number of topics for each entity. Once we know what are the topics that are talked about, we would like to know the sentiments inferred from the texts. This data can be extracted through a technique called [sentiment analysis](07SentimentAnalysis.ipynb).